# Практическое задание

Перевести ваш спарсенный json в формат csv таблицу через pandas

Для hh.ru:

* Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)
* Какая средняя и медианная зарплата по группам вакансий?
* Какая средняя и медианная зарплата по каждому региону?
* Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?
* Какое процентное соотношение каждого региона по вакансиям от всех вакансий?
* Какая корреляция уровня опыта от зарплаты?
* Сколько должностей в наборе данных?
* Какие 10 наиболее часто встречающихся должностей?

### Импортируем библиотеки

In [1]:
import json
import nltk
import numpy as np
import pandas as pd
import re

from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

### Загрузка json файлов

In [2]:
with open('../0_data_between3And6.json', 'r', encoding='utf-8') as file:
    data_between3And6 = json.load(file)
with open('../1_data_between1And3.json', 'r', encoding='utf-8') as file:
    data_between1And3 = json.load(file)
with open('../2_data_noExperience.json', 'r', encoding='utf-8') as file:
    data_noExperience = json.load(file)
with open('../3_data_moreThan6.json', 'r', encoding='utf-8') as file:
    data_moreThan6 = json.load(file)

In [3]:
data_list = []
for i in data_between3And6['data']:
    data_list.append([i['title'], i['work experience'], i['salary'], i['region']])
for i in data_between1And3['data']:
    data_list.append([i['title'], i['work experience'], i['salary'], i['region']])
for i in data_noExperience['data']:
    data_list.append([i['title'], i['work experience'], i['salary'], i['region']])
for i in data_moreThan6['data']:
    data_list.append([i['title'], i['work experience'], i['salary'], i['region']])

### Создание датафрейма

In [4]:
columns = data_moreThan6['data'][0].keys()

In [5]:
df = pd.DataFrame(data_list, columns=columns)

In [6]:
df.head()

,title,work experience,salary,region
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент


### Исследуем датафрейм

In [7]:
df.info

<bound method DataFrame.info of                                                   title work experience  \
0                 Backend разработчик со знанием Python   От 3 до 6 лет   
1                 Backend разработчик со знанием Python   От 3 до 6 лет   
2                 Backend разработчик со знанием Python   От 3 до 6 лет   
3                                   Backend разработчик   От 3 до 6 лет   
4                 Backend разработчик со знанием Python   От 3 до 6 лет   
...                                                 ...             ...   
4695               Senior System Engineer (Admins Team)     Более 6 лет   
4696               Senior System Engineer (Admins Team)     Более 6 лет   
4697                                    DevOps Engineer     Более 6 лет   
4698  Senior/Lead Software Engineer (.NET), Relocati...     Более 6 лет   
4699                    Risk Assurance Services Manager     Более 6 лет   

                 salary   region  
0     2 500 – 3 000 USD   Алматы

In [8]:
df.describe()

,title,work experience,salary,region
count,4700,4700,1636,4700
unique,2057,4,362,195
top,Специалист технической поддержки,От 3 до 6 лет,от 15 000 руб.,Москва
freq,133,2000,415,1980


### Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)

In [9]:
titles = df['title']
titles.head()

0    Backend разработчик со знанием Python
1    Backend разработчик со знанием Python
2    Backend разработчик со знанием Python
3                      Backend разработчик
4    Backend разработчик со знанием Python
Name: title, dtype: object

In [10]:
stemmer_rus = SnowballStemmer(language='russian')
stemmer_eng = SnowballStemmer(language='english')

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens_rus = []
    filtered_tokens_eng = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens_rus.append(token)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens_rus.append(token)
    stems_rus = [stemmer_rus.stem(t) for t in filtered_tokens_rus]
    stems_eng = [stemmer_eng.stem(t) for t in filtered_tokens_eng]
    stems = stems_rus + stems_eng
    return stems

In [11]:
stopwords = nltk.corpus.stopwords.words('russian')
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'со', 'senior', 'middle', 'middle+', 'ведущий', 'главный'])

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))

In [12]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(titles)

d:\python\de\1.1\venv\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'ведущ', 'впроч', 'всег', 'всегд', 'главн', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  warnings.warn(


CPU times: total: 922 ms
Wall time: 933 ms


In [13]:
type(tfidf_matrix)
tfidf_matrix

<4700x81 sparse matrix of type '<class 'numpy.float64'>'
	with 13723 stored elements in Compressed Sparse Row format>

In [14]:
tfidf_matrix.shape

(4700, 81)

In [15]:
num_clusters = 25
km = KMeans(n_clusters=num_clusters)

In [16]:
%%time
km.fit(tfidf_matrix)

CPU times: total: 2.72 s
Wall time: 502 ms


KMeans(n_clusters=25)

In [17]:
%%time
idx = km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: total: 2.64 s
Wall time: 448 ms


In [18]:
clusterkm = km.labels_.tolist()
frame = pd.DataFrame(titles)
out = {'title': titles, 'cluster': clusterkm}
frame = pd.DataFrame(out, columns = ['title', 'cluster'])

In [19]:
frame[frame.cluster.eq(0)]

,title,cluster
15,Разработчик Python,0
16,Разработчик Python,0
25,Разработчик Python (Senior/Principal),0
35,Python разработчик,0
41,Старший Python разработчик,0
...,...,...
3893,Python Junior разработчик,0
4489,Разработчик Python,0
4499,Senior Python разработчик,0
4515,Python разработчик,0


In [20]:
frame[frame.cluster.eq(1)]

,title,cluster
618,Senior QA Engineer (Python),1
678,Senior QA engineer / тестировщик ПО,1
774,Senior QA engineer (to Serbia/Canada),1
780,Senior QA engineer (to Serbia/Canada),1
813,QA Middle Engineer,1
...,...,...
3669,QA Engineer API/Web,1
3696,QA Engineer API/Web,1
3752,Тестировщик ПО (QA Engineer с уклоном в автома...,1
3797,QA middle engineer,1


In [21]:
frame[frame.cluster.eq(2)]

,title,cluster
101,R&D Программист Python (Computer Graphics prod...,2
118,Программист С++ (Middle / Senior),2
129,R&D Программист Python (Computer Graphics prod...,2
150,Программист разработчик SQL (SQL Developer),2
160,Программист С/С++ (Embedded Linux),2
...,...,...
4473,Начинающий программист,2
4477,Начинающий программист,2
4605,Программист C (Linux),2
4672,Инженер-электронщик / программист,2


In [22]:
frame[frame.cluster.eq(3)]

,title,cluster
9,Python developer (Middle),3
12,Python developer (Senior),3
19,Python Developer (Remote),3
20,Python Developer,3
23,Python Developer + ML,3
...,...,...
4518,Middle+/Senior Python Developer / Python разра...,3
4520,Senior Python developer (MLOps),3
4521,Senior python developer,3
4536,Senior / Lead Python developer (Private Aviati...,3


In [23]:
frame[frame.cluster.eq(4)]

,title,cluster
6,Разработчик Solidity/Разработчик Rust/Разработ...,4
27,Backend-разработчик,4
33,Ведущий Python-developer,4
37,Data-аналитик (ML),4
38,"Python Django разработчик, удаленно",4
...,...,...
4673,RTL-разработчик (Senior),4
4677,Старший специалист по верификации RTL (Senior),4
4689,Group Manager (Advanced Technologies and Perfo...,4
4694,CTO / Chief technical officer Robo Dev,4


In [24]:
frame[frame.cluster.eq(5)]

,title,cluster
11,Разработчик C# / Python,5
13,Python разработчик (личный кабинет пользователя),5
17,Python разработчик (дистанционно),5
22,Python/Django разработчик,5
26,Python разработчик (дистанционно),5
...,...,...
4527,Разработчик С++,5
4560,Senior .Net разработчик,5
4566,Senior .Net разработчик,5
4607,С++ разработчик,5


In [25]:
frame[frame.cluster.eq(6)]

,title,cluster
3911,Специалист службы поддержки с техническими зна...,6
3912,Специалист службы поддержки с техническими зна...,6
3913,Специалист службы поддержки с техническими зна...,6
3914,Специалист службы поддержки с техническими зна...,6
3961,Специалист службы поддержки с техническими зна...,6
...,...,...
4439,Специалист службы поддержки с техническими зна...,6
4440,Специалист службы поддержки с техническими зна...,6
4441,Специалист службы поддержки с техническими зна...,6
4442,Специалист службы поддержки с техническими зна...,6


In [26]:
frame[frame.cluster.eq(7)]

,title,cluster
300,Инженер-Программист (отдела функционального те...,7
318,Инженер-программист Python,7
387,Инженер-программист Python,7
476,Ведущий инженер-программист (разрабатывающее п...,7
553,Инженер-программист встраиваемых систем / Midd...,7
...,...,...
4455,Инженер-программист,7
4467,Инженер-программист,7
4553,Инженер-программист (Full Stack),7
4578,Ведущий инженер-программист по разработке сете...,7


In [27]:
frame[frame.cluster.eq(8)]

,title,cluster
8,"Python-разработчик (Django, DRF)",8
10,Python-разработчик,8
31,Fullstack Python-разработчик,8
34,Fullstack Python-разработчик,8
42,Senior Python-разработчик,8
...,...,...
4500,Middle/Senior Python-разработчик в команду Доб...,8
4501,Middle/Senior Python-разработчик в команду Доб...,8
4502,Middle/Senior Python-разработчик в команду Доб...,8
4504,Middle/Senior Python-разработчик в команду Доб...,8


In [28]:
frame[frame.cluster.eq(9)]

,title,cluster
142,Data Engineer,9
278,Data Engineer (middle+/senior),9
549,Data Engineer (ETL Developer),9
566,Data engineer,9
612,Разработчик курса/методист/преподаватель по Da...,9
...,...,...
4617,Data Science Эксперт в Yandex Cloud,9
4627,Senior Manager Data Analytics,9
4645,Data Science Эксперт в Yandex Cloud,9
4675,Архитектор Big Data,9


In [29]:
frame[frame.cluster.eq(10)]

,title,cluster
812,Инженер технической поддержки,10
1685,Специалист по техническому сопровождению инфор...,10
1745,PHP- разработчик с функциями технической подде...,10
2701,Инженер технической поддержки,10
2738,Специалист технической поддержки приложений дл...,10
...,...,...
4268,Специалист технической поддержки,10
4464,Специалист технической поддержки Yandex.SupportAl,10
4465,Специалист технической поддержки Yandex.SupportAl,10
4466,Специалист технической поддержки Yandex.SupportAl,10


In [30]:
frame[frame.cluster.eq(11)]

,title,cluster
439,Senior Python Engineer,11
474,Backend Engineer (Kotlin/C++/Python) at Joompa...,11
491,Backend Engineer (Kotlin/C++/Python) at Joompa...,11
519,Senior Python Engineer,11
532,Software Engineer C++ /Python,11
...,...,...
4687,Senior C++ Browser Engineer (relocation to EU),11
4693,Senior Software Engineer (Linux/С++),11
4695,Senior System Engineer (Admins Team),11
4696,Senior System Engineer (Admins Team),11


In [31]:
frame[frame.cluster.eq(12)]

,title,cluster
87,Pipeline developer / Пайплайн разработчик (Pyt...,12
123,Pyth‎on Developer,12
215,Фулл-стек разработчик / Full-stack developer,12
223,Back-End developer,12
232,R Developer,12
...,...,...
4637,Frontend developer (react),12
4663,"Senior Frontend Developer (Vue, Nuxt)",12
4665,Full-Stack Developer,12
4691,Full-stack Developer,12


In [32]:
frame[frame.cluster.eq(13)]

,title,cluster
673,Системный инженер (системный администратор),13
728,Ведущий системный аналитик,13
754,Системный архитектор,13
818,Системный аналитик (ИТ-разработка и консалтинг...,13
829,Системный архитектор,13
834,Системный аналитик,13
854,Системный аналитик,13
855,Системный аналитик (ИТ-разработка и консалтинг...,13
893,Системный Архитектор (Wink),13
1032,Системный аналитик (Wink),13


In [33]:
frame[frame.cluster.eq(14)]

,title,cluster
0,Backend разработчик со знанием Python,14
1,Backend разработчик со знанием Python,14
2,Backend разработчик со знанием Python,14
3,Backend разработчик,14
4,Backend разработчик со знанием Python,14
...,...,...
4498,Senior backend developer (Python),14
4503,Lead backend developer (Python),14
4519,Python Backend Engineer (ML Platform),14
4535,Senior Backend Developer (Python),14


In [34]:
frame[frame.cluster.eq(15)]

,title,cluster
514,Linux DevOps Engineer,15
517,Linux DevOps Engineer,15
575,DevOps,15
576,Senior DevOps Engineer (Yepp),15
579,DevOps engineer \ Девопс инженер (удалённо),15
...,...,...
4572,Senior Devops Engineer,15
4584,Solution Architect (DevOps),15
4585,Solution Architect (DevOps),15
4657,Lead DevOps,15


In [35]:
frame[frame.cluster.eq(16)]

,title,cluster
144,Automation QA (Java\Python),16
530,Automation QA (Python),16
531,QA Automation engineer (Python),16
557,Senior QA Automation (Python),16
563,Тестировщик (Automation engineer),16
...,...,...
4681,Senior QA Automaton (Limassol),16
4682,Senior QA Automaton (Limassol),16
4683,Senior QA Automaton (Limassol),16
4684,Senior QA Automaton (Limassol),16


In [36]:
frame[frame.cluster.eq(17)]

,title,cluster
5,Senior Data Scientist / ML Engineer,17
489,Data scientist / ML разработчик,17
545,Middle Data Scientist,17
585,Senior Data Scientist,17
606,Data Scientist (Core),17
...,...,...
3762,Data scientist,17
3819,Data Scientist,17
4539,Team lead Data Scientist,17
4540,Старший математик \ Senior Data scientist,17


In [37]:
frame[frame.cluster.eq(18)]

,title,cluster
559,Аналитик BI,18
887,Аналитик ЦФТ,18
895,Аналитик,18
913,Аналитик ЦФТ,18
918,Аналитик по распознаванию киберугроз,18
...,...,...
3999,Аналитик данных,18
4003,Аналитик SQL,18
4051,Аналитик SQL,18
4074,Младший аналитик (IT),18


In [38]:
frame[frame.cluster.eq(19)]

,title,cluster
7,Программист Python,19
18,"Программист Python, Django",19
66,Программист Python,19
79,Ведущий программист Python,19
103,Программист Python (middle),19
113,Программист Python,19
125,Программист Python,19
155,Программист Python (middle),19
174,Senior Python Программист,19
182,Ведущий программист Python,19


In [39]:
frame[frame.cluster.eq(20)]

,title,cluster
138,Team Lead Python,20
166,Python Team Lead / Руководитель backend разраб...,20
185,Python Team Lead / Руководитель backend разраб...,20
186,Python Team Lead,20
197,Python Team Lead,20
...,...,...
4644,Ведущий инженер-схемотехник (team lead),20
4667,Team Lead DevOps (Infrastructure),20
4668,Team Lead DevOps (Infrastructure),20
4670,Team Lead DevOps (DataOps Team) - Spain,20


In [40]:
frame[frame.cluster.eq(21)]

,title,cluster
490,Инженер по автоматизированному тестированию (B...,21
540,Инженер по автоматизированному тестированию (B...,21
555,DevOps инженер,21
584,DevOps инженер,21
590,Системный администратор Linux / DevOps инженер,21
...,...,...
4559,DevOps инженер (Openstack),21
4562,DevOps\SRE инженер,21
4563,DevOps\SRE инженер,21
4565,DevOps\SRE инженер,21


In [41]:
frame[frame.cluster.eq(22)]

,title,cluster
179,"Software research engineer (Java, Python)",22
208,"Software research engineer (Java, Python)",22
534,"Преподаватель IT курсов для взрослых (Python, ...",22
621,Программист Java (embedded-системы),22
675,Программист Java,22
677,Senior Java разработчик (Hospitality),22
739,Программист Java,22
752,Программист Java,22
844,Senior Java разработчик (Hospitality),22
884,Java Developer (middle),22


In [42]:
frame[frame.cluster.eq(23)]

,title,cluster
71,Data Analyst (middle/senior),23
210,Data analyst (Tableau),23
568,Старший аналитик данных / Senior Data Analyst,23
588,Data analyst (Tableau),23
620,Senior Product Analyst,23
...,...,...
4459,Senior Data Analyst,23
4530,Data Analyst - NLP (to Canada/Serbia),23
4533,Data Analyst - NLP (to Canada/Serbia),23
4624,Senior Geo analyst,23


In [43]:
frame[frame.cluster.eq(24)]

,title,cluster
260,"Backend Разработчик Golang, PHP / Python (mid+...",24
611,Senior Python / Golang developer,24
630,Golang Разработчик,24
720,Golang Разработчик,24
721,Golang разработчик,24
732,Golang Разработчик,24
745,Golang Разработчик,24
763,Senior Golang Developer,24
872,Middle/Senior Golang Developer,24
1035,Senior golang developer [3Сlick Закупки],24


In [44]:
df['category'] = frame['cluster']
df.head(10)

,title,work experience,salary,region,category
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14
5,Senior Data Scientist / ML Engineer,От 3 до 6 лет,None,Москва,17
6,Разработчик Solidity/Разработчик Rust/Разработ...,От 3 до 6 лет,от 600 000 руб.,Москва,4
7,Программист Python,От 3 до 6 лет,80 000 – 220 000 руб.,Москва,19
8,"Python-разработчик (Django, DRF)",От 3 до 6 лет,60 000 – 150 000 руб.,Москва,8
9,Python developer (Middle),От 3 до 6 лет,300 000 – 360 000 руб.,Санкт-Петербург,3


In [45]:
df.tail(10)

,title,work experience,salary,region,category
4690,Arch Lead,Более 6 лет,None,Москва,20
4691,Full-stack Developer,Более 6 лет,от 6 500 USD,Москва,12
4692,Senior Fullstack Developer,Более 6 лет,None,Москва,12
4693,Senior Software Engineer (Linux/С++),Более 6 лет,None,Москва,11
4694,CTO / Chief technical officer Robo Dev,Более 6 лет,300 000 – 350 000 руб.,Москва,4
4695,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Чехия,11
4696,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Армения,11
4697,DevOps Engineer,Более 6 лет,3 700 – 4 700 USD,Минск,15
4698,"Senior/Lead Software Engineer (.NET), Relocati...",Более 6 лет,3 000 – 5 500 USD,Армения,11
4699,Risk Assurance Services Manager,Более 6 лет,None,Ташкент,4


In [46]:
category_name = {
    0:'Разработчик Python',
    1:'QA engineer',
    2:'C++',
    3:'Python developer',
    4:'Другое',
    5:'Разработчик',
    6:'Специалист службы поддержки',
    7:'Инженер-программист',
    8:'Python разработчик',
    9:'Data Engineer',
    10:'Специалист технической поддержки',
    11:'Engineer',
    12:'Developer',
    13:'Системный аналитик',
    14:'Backend разработчик',
    15:'DevOps engineer',
    16:'QA Automation Engineer',
    17:'Data Scientist',
    18:'Аналитик',
    19:'Программист',
    20:'Тимлид',
    21:'Инженер',
    22:'Java',
    23:'Data Analyst',
    24:'Golang разработчик'
}

In [47]:
df['category_name'] = df['category'].map(category_name)
df.head()

,title,work experience,salary,region,category,category_name
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик


### Какая средняя и медианная зарплата по группам вакансий?

In [48]:
df.head(10)

,title,work experience,salary,region,category,category_name
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик
5,Senior Data Scientist / ML Engineer,От 3 до 6 лет,None,Москва,17,Data Scientist
6,Разработчик Solidity/Разработчик Rust/Разработ...,От 3 до 6 лет,от 600 000 руб.,Москва,4,Другое
7,Программист Python,От 3 до 6 лет,80 000 – 220 000 руб.,Москва,19,Программист
8,"Python-разработчик (Django, DRF)",От 3 до 6 лет,60 000 – 150 000 руб.,Москва,8,Python разработчик
9,Python developer (Middle),От 3 до 6 лет,300 000 – 360 000 руб.,Санкт-Петербург,3,Python developer


In [49]:
df.salary.unique()

array(['2 500 – 3 000 USD', None, 'от 600 000 руб.',
       '80 000 – 220 000 руб.', '60 000 – 150 000 руб.',
       '300 000 – 360 000 руб.', '120 000 – 190 000 руб.',
       '300 000 – 500 000 руб.', '360 000 – 480 000 руб.',
       '150 000 – 150 000 руб.', 'до 300 000 руб.', 'до 200 000 руб.',
       'от 110 000 руб.', '200 000 – 350 000 руб.', 'от 150 000 руб.',
       '280 000 – 340 000 руб.', 'от 3 500 USD', '150 000 – 200 000 руб.',
       '220 000 – 340 000 руб.', '200 000 – 300 000 руб.',
       '180 000 – 250 000 руб.', 'до 400 000 руб.', 'от 200 000 руб.',
       '3 000 – 5 000 USD', '200 000 – 400 000 руб.', '5 000 – 7 000 EUR',
       '100 000 – 250 000 руб.', '2 500 – 3 500 USD', 'до 250 000 руб.',
       '220 000 – 300 000 руб.', 'от 225 000 руб.', 'до 7 000 USD',
       'до 4 000 USD', 'до 220 000 руб.', 'до 6 000 USD',
       'от 250 000 руб.', '40 000 – 80 000 руб.', 'до 50 000 000 сум',
       'от 180 000 руб.', '4 000 – 5 700 EUR', 'от 140 000 руб.',
       'от 120

In [50]:
df['salary'].isnull().sum()

3064

In [51]:
currencies = {}
for i in df.salary:
    if i is not None:
        cur = i.split(' ')[-1]
        cur = cur.replace('\xa0', '')
        if cur not in currencies:
            currencies[cur] = 1
        else:
            currencies[cur] = currencies[cur] + 1
currencies

{'USD': 122, 'руб.': 1396, 'EUR': 82, 'сум': 2, 'KZT': 30, 'бел.руб.': 4}

In [52]:
exchange_rate = {
    'USD': 58.18,
    'руб.': 1,
    'EUR': 56,
    'сум': 0.00527,
    'KZT': 0.1219,
    'бел.руб.': 23.46
}

In [53]:
salary_from_national_currency = []
salary_from = []
salary_to_national_currency = []
salary_to = []
currency = []
for i in df.salary:
    if i is None:
        salary_from.append(None)
        salary_from_national_currency.append(None)
        salary_to.append(None)
        salary_to_national_currency.append(None)
    else:
        cur = i.split(' ')[-1]
        cur = cur.replace('\xa0', '')
        currency.append(cur)
        string = i.replace('\xa0', '').replace(' ', '').rstrip(cur)
        if '–' in string:
            _from, _to = list(map(int,string.split('–')))
            salary_from_national_currency.append(_from)
            salary_to_national_currency.append(_to)
            salary_from.append(_from*exchange_rate[cur])
            salary_to.append(_to*exchange_rate[cur])
        elif 'до' in string:
            _to = int(string.lstrip('до'))
            salary_from_national_currency.append(_to)
            salary_to_national_currency.append(_to)
            salary_from.append(_to*exchange_rate[cur])
            salary_to.append(_to*exchange_rate[cur])
        elif 'от' in string:
            _from = int(string.lstrip('от'))
            salary_from_national_currency.append(_from)
            salary_to_national_currency.append(_from)
            salary_from.append(_from*exchange_rate[cur])
            salary_to.append(_from*exchange_rate[cur])
        else:
            salary_from.append(None)
            salary_from_national_currency.append(None)
            salary_to.append(None)
            salary_to_national_currency.append(None)

In [54]:
df_salary_category = df.copy(deep=True)
df_salary_category['from_RUB'] = salary_from
df_salary_category['to_RUB'] = salary_to
df_salary_category['from_local_cur'] = salary_from_national_currency
df_salary_category['to_local_cur'] = salary_to_national_currency
df_salary_category['RUB'] = (df_salary_category['from_RUB']+df_salary_category['to_RUB'])//2
df_salary_category

,title,work experience,salary,region,category,category_name,from_RUB,to_RUB,from_local_cur,to_local_cur,RUB
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
...,...,...,...,...,...,...,...,...,...,...,...
4695,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Чехия,11,Engineer,191994.0,191994.0,3300.0,3300.0,191994.0
4696,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Армения,11,Engineer,191994.0,191994.0,3300.0,3300.0,191994.0
4697,DevOps Engineer,Более 6 лет,3 700 – 4 700 USD,Минск,15,DevOps engineer,215266.0,273446.0,3700.0,4700.0,244356.0
4698,"Senior/Lead Software Engineer (.NET), Relocati...",Более 6 лет,3 000 – 5 500 USD,Армения,11,Engineer,174540.0,319990.0,3000.0,5500.0,247265.0


In [55]:
df_salary_category.RUB.mean()

132691.0770171149

In [56]:
df_salary_category[['category_name', 'RUB']].groupby('category_name').describe()

RUB                                \
                                  count           mean            std   
category_name                                                           
Backend разработчик                63.0  202010.000000   94542.634029   
C++                               111.0  122297.783784   78769.018792   
Data Analyst                       20.0  215855.750000   91118.171613   
Data Engineer                      26.0  283642.192308   82391.293789   
Data Scientist                     11.0  230909.090909  102122.920586   
DevOps engineer                    56.0  298921.178571   78596.366006   
Developer                          83.0  190779.469880  101126.781155   
Engineer                           40.0  265901.150000   85230.825326   
Golang разработчик                  5.0  178090.000000   95258.151882   
Java                               13.0  203669.230769   84675.068985   
Python developer                  105.0  220705.200000   92138.783924   
Python разработчик                 22.0  184545.454545  101757.286985   
QA Automation Engineer             19.0  173807.052632   68672.613821   
QA engineer                        11.0  176319.545455   70116.543143   
Аналитик                           16.0  130000.000000   52788.887720   
Другое                            256.0  136987.375000   97561.532491   
Инженер                            89.0  156185.393258   54204.420632   
Инженер-программист                87.0  124437.678161   76098.722494   
Программист                        29.0  166174.206897   70064.775797   
Разработчик                        70.0  137256.214286   63275.385376   
Разработчик Python                 47.0  187472.340426  101651.774621   
Системный аналитик                  9.0  183888.888889  105142.099613   
Специалист службы поддержки       285.0   15000.000000       0.000000   
Специалист технической поддержки  154.0   20929.870130   22138.146565   
Тимлид                              9.0  339777.777778   81356.895494   

                                                                           \
                                       min       25%       50%        75%   
category_name                                                               
Backend разработчик                24380.0  150000.0  200000.0  271600.00   
C++                                17500.0   62500.0  103900.0  168750.00   
Data Analyst                       95000.0  168000.0  200000.0  214538.75   
Data Engineer                      76967.0  236065.0  336000.0  336000.00   
Data Scientist                    120000.0  150000.0  170000.0  325000.00   
DevOps engineer                   159995.0  252000.0  280000.0  314083.75   
Developer                          18285.0  118180.0  170000.0  250000.00   
Engineer                          159995.0  196000.0  239992.5  300497.50   
Golang разработчик                 45000.0  145450.0  196000.0  196000.00   
Java                               60000.0  170000.0  190000.0  290900.00   
Python developer                    7250.0  165000.0  232720.0  275000.00   
Python разработчик                 40000.0  110000.0  155000.0  268750.00   
QA Automation Engineer            100000.0  120000.0  140000.0  231360.00   
QA engineer                       100000.0  115950.0  174540.0  196000.00   
Аналитик                           60000.0  103750.0  120000.0  151250.00   
Другое                             10000.0   56875.0  125000.0  183821.75   
Инженер                            85000.0  120000.0  135000.0  200000.00   
Инженер-программист                17500.0   67500.0  105000.0  188504.00   
Программист                        45000.0  107552.0  180000.0  225000.00   
Разработчик                        20000.0  108125.0  125950.0  157500.00   
Разработчик Python                 30000.0  110000.0  150000.0  250000.00   
Системный аналитик                 50000.0  120000.0  170000.0  175000.00   
Специалист службы поддержки        15000.0   15000.0   15000.0   1

In [57]:
df_salary_category_mean = pd.pivot_table(df_salary_category[['category_name', 'RUB']], index=['category_name'])
df_salary_category_mean

,RUB
category_name,
Backend разработчик,202010.000000
C++,122297.783784
Data Analyst,215855.750000
Data Engineer,283642.192308
Data Scientist,230909.090909
DevOps engineer,298921.178571
Developer,190779.469880
Engineer,265901.150000
Golang разработчик,178090.000000


In [58]:
df_salary_category_median = pd.pivot_table(df_salary_category[['category_name', 'RUB']], index=['category_name'], aggfunc=[np.median])
df_salary_category_median

,median
,RUB
category_name,
Backend разработчик,200000.0
C++,103900.0
Data Analyst,200000.0
Data Engineer,336000.0
Data Scientist,170000.0
DevOps engineer,280000.0
Developer,170000.0
Engineer,239992.5


In [59]:
df_salary_category_mean_median = pd.merge(df_salary_category_mean, df_salary_category_median, left_index=True,right_index=True )
df_salary_category_mean_median

C:\Users\litvinov_ny\AppData\Local\Temp\ipykernel_11300\3081404117.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  df_salary_category_mean_median = pd.merge(df_salary_category_mean, df_salary_category_median, left_index=True,right_index=True )


,RUB,"(median, RUB)"
category_name,,
Backend разработчик,202010.000000,200000.0
C++,122297.783784,103900.0
Data Analyst,215855.750000,200000.0
Data Engineer,283642.192308,336000.0
Data Scientist,230909.090909,170000.0
DevOps engineer,298921.178571,280000.0
Developer,190779.469880,170000.0
Engineer,265901.150000,239992.5
Golang разработчик,178090.000000,196000.0


### Какая средняя и медианная зарплата по каждому региону?

In [60]:
df.head(10)

,title,work experience,salary,region,category,category_name
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик
5,Senior Data Scientist / ML Engineer,От 3 до 6 лет,None,Москва,17,Data Scientist
6,Разработчик Solidity/Разработчик Rust/Разработ...,От 3 до 6 лет,от 600 000 руб.,Москва,4,Другое
7,Программист Python,От 3 до 6 лет,80 000 – 220 000 руб.,Москва,19,Программист
8,"Python-разработчик (Django, DRF)",От 3 до 6 лет,60 000 – 150 000 руб.,Москва,8,Python разработчик
9,Python developer (Middle),От 3 до 6 лет,300 000 – 360 000 руб.,Санкт-Петербург,3,Python developer


In [61]:
df.salary.unique()

array(['2 500 – 3 000 USD', None, 'от 600 000 руб.',
       '80 000 – 220 000 руб.', '60 000 – 150 000 руб.',
       '300 000 – 360 000 руб.', '120 000 – 190 000 руб.',
       '300 000 – 500 000 руб.', '360 000 – 480 000 руб.',
       '150 000 – 150 000 руб.', 'до 300 000 руб.', 'до 200 000 руб.',
       'от 110 000 руб.', '200 000 – 350 000 руб.', 'от 150 000 руб.',
       '280 000 – 340 000 руб.', 'от 3 500 USD', '150 000 – 200 000 руб.',
       '220 000 – 340 000 руб.', '200 000 – 300 000 руб.',
       '180 000 – 250 000 руб.', 'до 400 000 руб.', 'от 200 000 руб.',
       '3 000 – 5 000 USD', '200 000 – 400 000 руб.', '5 000 – 7 000 EUR',
       '100 000 – 250 000 руб.', '2 500 – 3 500 USD', 'до 250 000 руб.',
       '220 000 – 300 000 руб.', 'от 225 000 руб.', 'до 7 000 USD',
       'до 4 000 USD', 'до 220 000 руб.', 'до 6 000 USD',
       'от 250 000 руб.', '40 000 – 80 000 руб.', 'до 50 000 000 сум',
       'от 180 000 руб.', '4 000 – 5 700 EUR', 'от 140 000 руб.',
       'от 120

In [62]:
df['salary'].isnull().sum()

3064

In [63]:
df_salary_regions = df.copy(deep=True)
df_salary_regions['from_RUB'] = salary_from
df_salary_regions['to_RUB'] = salary_to
df_salary_regions['from_local_cur'] = salary_from_national_currency
df_salary_regions['to_local_cur'] = salary_to_national_currency
df_salary_regions['RUB'] = (df_salary_regions['from_RUB']+df_salary_regions['to_RUB'])//2
df_salary_regions

,title,work experience,salary,region,category,category_name,from_RUB,to_RUB,from_local_cur,to_local_cur,RUB
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик,145450.0,174540.0,2500.0,3000.0,159995.0
...,...,...,...,...,...,...,...,...,...,...,...
4695,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Чехия,11,Engineer,191994.0,191994.0,3300.0,3300.0,191994.0
4696,Senior System Engineer (Admins Team),Более 6 лет,от 3 300 USD,Армения,11,Engineer,191994.0,191994.0,3300.0,3300.0,191994.0
4697,DevOps Engineer,Более 6 лет,3 700 – 4 700 USD,Минск,15,DevOps engineer,215266.0,273446.0,3700.0,4700.0,244356.0
4698,"Senior/Lead Software Engineer (.NET), Relocati...",Более 6 лет,3 000 – 5 500 USD,Армения,11,Engineer,174540.0,319990.0,3000.0,5500.0,247265.0


In [64]:
df_salary_regions.RUB.mean()

132691.0770171149

In [65]:
df_salary_regions[['region', 'RUB']].groupby('region').describe()

RUB                                                          \
             count           mean           std      min      25%      50%   
region                                                                       
Абакан         4.0   31750.000000  14614.490526  15000.0  21750.0  34000.0   
Актау          1.0   58180.000000           NaN  58180.0  58180.0  58180.0   
Алматы        25.0  108898.120000  82635.731709  18285.0  29865.0  82282.0   
Альметьевск    1.0   15000.000000           NaN  15000.0  15000.0  15000.0   
Арзамас        2.0   15000.000000      0.000000  15000.0  15000.0  15000.0   
...            ...            ...           ...      ...      ...      ...   
Электросталь   1.0   15000.000000           NaN  15000.0  15000.0  15000.0   
Элиста         1.0   15000.000000           NaN  15000.0  15000.0  15000.0   
Энгельс        1.0   15000.000000           NaN  15000.0  15000.0  15000.0   
Якутск         0.0            NaN           NaN      NaN      NaN      NaN   
Ярославль     19.0   96126.315789  75834.526306  15000.0  19700.0  90000.0   

                                  
                   75%       max  
region                            
Абакан         44000.0   44000.0  
Актау          58180.0   58180.0  
Алматы        159995.0  305445.0  
Альметьевск    15000.0   15000.0  
Арзамас        15000.0   15000.0  
...                ...       ...  
Электросталь   15000.0   15000.0  
Элиста         15000.0   15000.0  
Энгельс        15000.0   15000.0  
Якутск             NaN       NaN  
Ярославль     167500.0  225000.0  

[195 rows x 8 columns]

In [66]:
pd.pivot_table(df_salary_regions[['region', 'RUB']], index=['region'])

,RUB
region,
Абакан,31750.000000
Актау,58180.000000
Алматы,108898.120000
Альметьевск,15000.000000
Арзамас,15000.000000
...,...
Щелково,33333.333333
Электросталь,15000.000000
Элиста,15000.000000


In [67]:
pd.pivot_table(df_salary_regions[['region', 'RUB']], index=['region'], aggfunc=[np.median])

,median
,RUB
region,
Абакан,34000.0
Актау,58180.0
Алматы,82282.0
Альметьевск,15000.0
Арзамас,15000.0
...,...
Щелково,15000.0
Электросталь,15000.0


### Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?

In [68]:
df_salary_category_mean.sort_values('RUB', ascending=False)

,RUB
category_name,
Тимлид,339777.777778
DevOps engineer,298921.178571
Data Engineer,283642.192308
Engineer,265901.150000
Data Scientist,230909.090909
Python developer,220705.200000
Data Analyst,215855.750000
Java,203669.230769
Backend разработчик,202010.000000


In [69]:
df_salary_category_mean[df_salary_category_mean.RUB == max(df_salary_category_mean.RUB)]

,RUB
category_name,
Тимлид,339777.777778


### Какое процентное соотношение каждого региона по вакансиям от всех вакансий?

In [70]:
df.head()

,title,work experience,salary,region,category,category_name
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик


In [71]:
df['region'].value_counts().head()

Москва             1980
Санкт-Петербург     637
Новосибирск         162
Нижний Новгород     127
Екатеринбург        126
Name: region, dtype: int64

In [72]:
region_count = df['region'].value_counts().to_dict()

In [73]:
region_count

{'Москва': 1980,
 'Санкт-Петербург': 637,
 'Новосибирск': 162,
 'Нижний Новгород': 127,
 'Екатеринбург': 126,
 'Минск': 88,
 'Казань': 84,
 'Алматы': 74,
 'Самара': 61,
 'Тбилиси': 57,
 'Краснодар': 53,
 'Нур-Султан': 49,
 'Томск': 44,
 'Ростов-на-Дону': 44,
 'Пермь': 44,
 'Ташкент': 44,
 'Уфа': 39,
 'Кипр': 39,
 'Челябинск': 32,
 'Воронеж': 32,
 'Волгоград': 28,
 'Омск': 27,
 'Красноярск': 26,
 'Армения': 25,
 'Калининград': 25,
 'Ярославль': 24,
 'Ижевск': 23,
 'Саратов': 22,
 'Владивосток': 19,
 'Рязань': 18,
 'Зеленоград': 18,
 'Сочи': 16,
 'Иркутск': 16,
 'Тюмень': 16,
 'Тула': 16,
 'Пенза': 13,
 'Кемерово': 13,
 'Хабаровск': 13,
 'Севастополь': 11,
 'Ульяновск': 11,
 'Йошкар-Ола': 11,
 'Таганрог': 9,
 'Оренбург': 9,
 'Белгород': 9,
 'Сербия': 9,
 'Батуми': 8,
 'Набережные Челны': 8,
 'Магнитогорск': 8,
 'Люберцы': 8,
 'Вологда': 8,
 'Тольятти': 8,
 'Болгария': 8,
 'Химки': 7,
 'Тверь': 7,
 'Иваново (Ивановская область)': 7,
 'Череповец': 7,
 'Саранск': 7,
 'Симферополь': 7,
 'Бря

In [74]:
total = df.title.count()
total

4700

In [75]:
df_percentage_regions = df.copy(deep=True)
df_percentage_regions['percentage region'] = df['region'].map(region_count) / total * 100

In [76]:
df_percentage_regions.head(10)

,title,work experience,salary,region,category,category_name,percentage region
0,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Алматы,14,Backend разработчик,1.574468
1,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Баку,14,Backend разработчик,0.106383
2,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Бишкек,14,Backend разработчик,0.106383
3,Backend разработчик,От 3 до 6 лет,2 500 – 3 000 USD,Минск,14,Backend разработчик,1.872340
4,Backend разработчик со знанием Python,От 3 до 6 лет,2 500 – 3 000 USD,Ташкент,14,Backend разработчик,0.936170
5,Senior Data Scientist / ML Engineer,От 3 до 6 лет,None,Москва,17,Data Scientist,42.127660
6,Разработчик Solidity/Разработчик Rust/Разработ...,От 3 до 6 лет,от 600 000 руб.,Москва,4,Другое,42.127660
7,Программист Python,От 3 до 6 лет,80 000 – 220 000 руб.,Москва,19,Программист,42.127660
8,"Python-разработчик (Django, DRF)",От 3 до 6 лет,60 000 – 150 000 руб.,Москва,8,Python разработчик,42.127660
9,Python developer (Middle),От 3 до 6 лет,300 000 – 360 000 руб.,Санкт-Петербург,3,Python developer,13.553191


In [77]:
df_percentage_regions[['region', 'percentage region']].value_counts()

region                     percentage region
Москва                     42.127660            1980
Санкт-Петербург            13.553191             637
Новосибирск                3.446809              162
Нижний Новгород            2.702128              127
Екатеринбург               2.680851              126
                                                ... 
Северск (Томская область)  0.021277                1
Гомель                     0.021277                1
Сергиев Посад              0.021277                1
Серпухов                   0.021277                1
Копейск                    0.021277                1
Length: 195, dtype: int64

### Какая корреляция уровня опыта от зарплаты?

In [78]:
df_cor = pd.DataFrame(df_salary_regions, columns=['work experience', 'RUB'])
df_cor.head()

,work experience,RUB
0,От 3 до 6 лет,159995.0
1,От 3 до 6 лет,159995.0
2,От 3 до 6 лет,159995.0
3,От 3 до 6 лет,159995.0
4,От 3 до 6 лет,159995.0


In [79]:
df_cor =df_cor.replace({'work experience': {'Нет опыта':0,'От 1 года до 3 лет':1,'От 3 до 6 лет':2,'Более 6 лет':3}})
df_cor.head()

,work experience,RUB
0,2,159995.0
1,2,159995.0
2,2,159995.0
3,2,159995.0
4,2,159995.0


In [80]:
df_cor.corr()

,work experience,RUB
work experience,1.00000,0.74834
RUB,0.74834,1.00000


In [81]:
df_cor.corr().loc['work experience', 'RUB']

0.7483399266868055

### Сколько должностей в наборе данных?

In [82]:
df['title'].nunique()

2057

### Какие 10 наиболее часто встречающихся должностей?

In [83]:
df[['title']].value_counts().head(10)

title                                                                 
Специалист технической поддержки                                          133
Специалист службы поддержки с техническими знаниями                        87
Специалист службы поддержки с техническими знаниями в Яндекс Поиск         83
Python Developer                                                           48
Специалист службы поддержки с техническими знаниями (Умные устройства)     46
Инженер-программист                                                        41
Programming Team Lead (С++)                                                37
Разработчик Python                                                         35
Специалист службы поддержки с техническими знаниями (Контест)              32
Программист Python                                                         32
dtype: int64